# Optimization of Microgrid with long-term Hydrogen storage : Comparison of 2 rule based strategy
This work have been undertaken using the Package [Microgrid.jl H2 tree](https://github.com/nikiema-fruc/Microgrids.jl/tree/h2) .
 The aim is to conduct a techno-economic comparison between 2 rule based operation strategies applied to an islanded Microgrid with Hydrogen long term storage system. 

<img alt='schematic of a microgrid with a symbolic representation of the notion of sizing' src='./figures/interwined_sizing_operation_problem.png' style='height:18em'>

In [3]:
using CSV, DataFrames
using Microgrids
using NLopt # optimization solvers
using Printf # pretty print results
using Random, Statistics
using PyPlot
using BenchmarkTools # for timing performance, optional

In [4]:
with_plots = isdefined(Main, :PyPlot)

true

## Load Microgrid Project data
Loading parameters and time series for a Microgrid project with wind and solar sources, plus a battery and Hydrogen related components. 

In [5]:
include("../examples/data/Microgrid_Wind-Solar-H2_data.jl");

Data definition for Microgrid with wind, solar and Hybrid Storage System...


## Useful functions


In [6]:
"""
    Create a Microgrid of size `x`
    with x=[energy_rated_sto, power_rated_pv, power_rated_wind, power_rated_elyz, power_rated_fc, capacity_rated_hy_tank]
    You can also specify capex prices and initial filling rate of either Hydrogen tank or batteries.
    new_microgrid(x::Vector{Float64}= X,capex::Vector{Float64}=capex_def,initial_fill_rate::Vector{Float64})

"""

function new_microgrid(x::Vector{Float64}= X,capex::Vector{Float64}=capex_def,initial_fill_rate::Vector{Float64}=ini_filling_state)
    project = Project(lifetime, discount_rate, timestep, "€")
  gen = ProductionUnit(0.0,
      fuel_intercept, fuel_slope, fuel_price,
      capex[1], om_price_gen, lifetime_gen_y,lifetime_gen_h,
      load_ratio_min_gen, replacement_price_ratio,
      salvage_price_ratio, input_unit_gen,output_unit_gen)
  ftank = Tank(capacity_rated_ftank,capex[2], om_price_ftank,lifetime_ftank,loss_factor_ftank,initial_fill_rate[1],
          fuel_min_ratio, fuel_max_ratio,fuel_price, replacement_price_ratio, salvage_price_ratio)
  fuel_cell = ProductionUnit(x[5]*1000,cons_intercept_fc, cons_rate_fc,cons_price_fc,capex[3], om_price_fc,lifetime_fc_y,lifetime_fc_h,
              load_min_ratio_fc,replacement_price_ratio, salvage_price_ratio,input_unit_fc,output_unit_fc)
  hytank = Tank(x[6]*1000,capex[4], om_price_hytank,lifetime_hytank,loss_factor_hytank,initial_fill_rate[2],
              LoH_min_ratio, LoH_max_ratio,hy_price,replacement_price_ratio, salvage_price_ratio)
  dispatchables = DispatchableCompound{Float64}([gen], [fuel_cell])
     tanks = TankCompound{Float64}(ftank,hytank)

  elyz = ProductionUnit(x[4]*1000,cons_intercept_elyz,cons_slope_elyz,cons_price_elyz, capex[5], om_price_elyz, lifetime_elyz_y,lifetime_elyz_h,
  load_min_ratio_elyz,replacement_price_ratio, salvage_price_ratio,input_unit_elyz,output_unit_elyz)

  batt = Battery(x[1]*1000,
      capex[6], om_price_sto, lifetime_sto, lifetime_cycles,
      charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_max,initial_fill_rate[3],
      replacement_price_ratio, salvage_price_ratio)
  pv = Photovoltaic(x[2]*1000, irradiance,
      capex[7], om_price_pv,
      lifetime_pv, derating_factor_pv,
      replacement_price_ratio, salvage_price_ratio)
  windgen = WindPower(x[3]*1000, cf_wind,
      capex[8], om_price_wind,
      lifetime_wind,
      replacement_price_ratio, salvage_price_ratio)
  
 mg = Microgrid(project, Pload,dispatchables,
  [elyz,],tanks,batt, [
      pv,
      windgen
      ])
  return mg
  end

new_microgrid (generic function with 4 methods)

In [7]:
"""Simulate the performance of a Microgrid project 
Returns mg, traj, stats, costs
"""
    function simulate_microgrid(x=X,capex=capex_def,dispatch=dispatch_1)
     mg=new_microgrid(x)
    # Split decision variables (converted MW → kW):
    oper_traj = operation(mg, dispatch)
    if mg.tanks.h2Tank.capacity>0.0
        a = oper_traj.LoH[end]/mg.tanks.h2Tank.capacity
    else
        a=0.0
    end
   
    if mg.storage.energy_rated >0.0
         b = oper_traj.Ebatt[end]/mg.storage.energy_rated
    else
        b=0.0
    end
    
    ini=[0.0,a,b]
    mg=new_microgrid(x,capex,ini)
    # Launch simulation:
    traj, stats, costs = simulate(mg,dispatch)

    return  mg, traj ,stats, costs
end

simulate_microgrid

# Optimization Setting up

#### setting up the cost function

In [8]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x= X,capex=capex_def,dispatch=dispatch_1)
    mg,traj,stats, costs = simulate_microgrid(x,capex,dispatch)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [9]:
"""
Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`
with signature adapted to NLopt with `grad` as 2nd argument
load shedding penalty threshold `shed_max` should be in  [0,1[
"""
function obj(x,grad, shed_max,capex=capex_def,dispatch=dispatch_1, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x,capex,dispatch)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj

##### Test the objective function

In [10]:
"with default values"
mg,traj,stats,costs = simulate_microgrid()
X, costs.lcoe, costs.npc/1e6, stats.shed_rate

([5.0, 3.0, 1.8, 2.0, 1.8, 10.0], 0.3484277175377336, 32.55545241291073, 0.021478152526043473)

In [11]:
# Test:
shed_max = 0.01 # in [0,1]

println("Base. multi: ", obj_multi(X), " mono: ", obj(X,[], shed_max))

Base. multi: (0.3484277175377336, 0.021478152526043473) mono: 1148.163680321885


#### setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [12]:
Pload_max = maximum(Pload)

 xmin = [0., 1e-3, 0., 0., 0., 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
 x0 = X 
 xmax = [10.0, 10.0, 1800/Pload_max, 5., 2. , 15. ] * (Pload_max/1000) # In our case of study Wind capacity is limited to 2 Turbines of 900 kW

6-element Vector{Float64}:
 17.07
 17.07
  1.8
  8.535
  3.414
 25.605

Check cost function on xmin and xmax

In [13]:

obj_multi(xmin), obj_multi(xmax)

((0.10149685980963595, 0.9998470957371233), (0.9443504188446606, 0.0))

#### Wrapper of the optimization process

In [14]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0,capex, shed_max,dispatch, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-8, srand=1,w_shed=1e5)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad, shed_max,capex,dispatch)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = NLopt.optimize(opt, x0)
    return xopt, ret, opt.numevals
end

optim_mg

## 1- comparison of the use of these 2 strategies on optimal Sizing
The strategies to be applied are priority based. Strategy 1 will use the battery before the hydrogen chain to store excess energy or power the load in the event of a lack of renewable generation. Strategy 2 do the opposite by using primarily the power-H2-power chain. Flowcharts could be found in *images* folder.


In [26]:
df1 = DataFrame( dispatch=Int64[],xopt_pv=Float64[],xopt_wind=Float64[] ,xopt_sto=Float64[],xopt_elyz=Float64[], xopt_fc=Float64[], xopt_hyTank=Float64[],
    lcoe_opt=Float64[], shed_max=Float64[],algo=String[],srand=Int[], ret=String[],numevals=Int64[],time=Float64[],shed_rate_opt=Float64[],
    capex=Float64[],om=Float64[],replacement=Float64[],opex=Float64[],
    fc_hours=Float64[],elyz_hours=Float64[],fc_starts=Float64[],elyz_starts=Float64[],batt_cycles=Float64[],h2_consumed=Float64[],ini_h2_level=Float64[],ini_bat_level=Float64[],
       fc_energy=Float64[],batt_energy=Float64[]
    
)

Row,dispatch,xopt_pv,xopt_wind,xopt_sto,xopt_elyz,xopt_fc,xopt_hyTank,lcoe_opt,shed_max,algo,srand,ret,numevals,time,shed_rate_opt,capex,om,replacement,opex,fc_hours,elyz_hours,fc_starts,elyz_starts,batt_cycles,h2_consumed,ini_h2_level,ini_bat_level,fc_energy,batt_energy
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Int64,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [23]:
nrand = 10
shed_max_list=collect(0.0:0.00005:0.001)
srand_list = collect(1:nrand)
dispatchlist=[dispatch_1,dispatch_2]

2-element Vector{Function}:
 dispatch_1 (generic function with 2 methods)
 dispatch_2 (generic function with 1 method)

#### Loop to collect data for comparision
This loop will allow to to obtain optimization results for each strategy , depend on the shed_max. We will also conduct the optimization using different seeds to ensure the consistency of the convergence.

In [27]:
 algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
pareto_lcoe1 = zeros(Float64,21)
pareto_lcoe2 = zeros(Float64,21)
maxeval=100000
xtol_rel=1e-8
i=0
for shed_max in shed_max_list , srand in srand_list, dispatch in dispatchlist
      t_ini= time()
    xopt, ret, numevals = optim_mg(x0,capex_def, shed_max,dispatch, algo, maxeval,xtol_rel,srand)
    topt = time() - t_ini
    mg,traj,stats,costs = simulate_microgrid(xopt,capex_def,dispatch)
    
    push!(df1,(findfirst(==(dispatch),dispatchlist),xopt[2],xopt[3], xopt[1],xopt[4], xopt[5], xopt[6],
            costs.lcoe,shed_max,"CRS2",srand,String(ret),numevals,topt,stats.shed_rate,
         costs.system.investment,costs.system.om,costs.system.replacement, costs.system.om+costs.system.replacement,stats.fc_hours,stats.elyz_hours,stats.fc_starts,stats.elyz_starts,stats.storage_cycles,
            stats.h2_consumed, mg.tanks.h2Tank.ini_filling_ratio,mg.storage.SoC_ini,stats.fc_energy,stats.storage_dis_energy     
        ))
        i=i+1
     @printf("%s algo: %s after %d iterations. %d \nx*=", algo, ret,numevals,i)
println(round.(xopt*1000; digits=6)) # kW
end

GN_CRS2_LM algo: XTOL_REACHED after 16719 iterations. 1 
x*=[5488.787623, 4001.076852, 1800.0, 865.828862, 1536.749704, 16270.322139]
GN_CRS2_LM algo: XTOL_REACHED after 13607 iterations. 2 
x*=[262.366327, 6680.863763, 1799.999992, 1901.24334, 1423.098541, 16999.930026]
GN_CRS2_LM algo: XTOL_REACHED after 20023 iterations. 3 
x*=[5488.784123, 4001.077542, 1800.0, 865.829094, 1536.749702, 16270.322635]
GN_CRS2_LM algo: XTOL_REACHED after 13998 iterations. 4 
x*=[262.366833, 6680.863135, 1800.0, 1901.243421, 1423.098428, 16999.929527]
GN_CRS2_LM algo: XTOL_REACHED after 17973 iterations. 5 
x*=[5488.779564, 4001.077165, 1800.0, 865.829847, 1536.749717, 16270.324667]
GN_CRS2_LM algo: XTOL_REACHED after 13763 iterations. 6 
x*=[262.369497, 6680.863847, 1800.0, 1901.243213, 1423.098041, 16999.928922]
GN_CRS2_LM algo: XTOL_REACHED after 18391 iterations. 7 
x*=[5488.780355, 4001.077538, 1799.999998, 865.829638, 1536.749706, 16270.323903]
GN_CRS2_LM algo: XTOL_REACHED after 13535 iterations.

In [29]:
df1

Row,dispatch,xopt_pv,xopt_wind,xopt_sto,xopt_elyz,xopt_fc,xopt_hyTank,lcoe_opt,shed_max,algo,srand,ret,numevals,time,shed_rate_opt,capex,om,replacement,opex,fc_hours,elyz_hours,fc_starts,elyz_starts,batt_cycles,h2_consumed,ini_h2_level,ini_bat_level,fc_energy,batt_energy
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Int64,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,4.00108,1.8,5.48879,0.865829,1.53675,16.2703,0.359156,0.0,CRS2,1,XTOL_REACHED,16719,16.624,0.0,2.50048e7,6.86715e6,2.85743e6,9.72458e6,1200.0,3583.0,125.0,233.0,161.902,42948.9,0.0140035,0.0,6.87182e5,8.44215e5
2,2,6.68086,1.8,0.262366,1.90124,1.4231,16.9999,0.413641,0.0,CRS2,1,XTOL_REACHED,13607,13.286,0.0,2.82309e7,7.49706e6,4.29165e6,1.17887e7,2897.0,4820.0,309.0,350.0,39.6973,89869.8,0.013255,0.160586,1.43792e6,9894.47
3,1,4.00108,1.8,5.48878,0.865829,1.53675,16.2703,0.359156,0.0,CRS2,2,XTOL_REACHED,20023,18.078,0.0,2.50048e7,6.86715e6,2.85743e6,9.72458e6,1200.0,3583.0,125.0,233.0,161.902,42948.9,0.0140035,0.0,687182.0,844215.0
4,2,6.68086,1.8,0.262367,1.90124,1.4231,16.9999,0.413641,0.0,CRS2,2,XTOL_REACHED,13998,12.96,0.0,2.82309e7,7.49706e6,4.29165e6,1.17887e7,2897.0,4820.0,309.0,350.0,39.6973,89869.8,0.0132549,0.160586,1.43792e6,9894.49
5,1,4.00108,1.8,5.48878,0.86583,1.53675,16.2703,0.359156,0.0,CRS2,3,XTOL_REACHED,17973,16.661,0.0,2.50048e7,6.86715e6,2.85743e6,9.72458e6,1200.0,3583.0,125.0,233.0,161.902,42948.9,0.0140035,0.0,6.87182e5,8.44215e5
6,2,6.68086,1.8,0.262369,1.90124,1.4231,16.9999,0.413641,0.0,CRS2,3,XTOL_REACHED,13763,12.946,0.0,2.82309e7,7.49706e6,4.29165e6,1.17887e7,2897.0,4820.0,309.0,350.0,39.6969,89869.8,0.0132549,0.16059,1.43792e6,9894.5
7,1,4.00108,1.8,5.48878,0.86583,1.53675,16.2703,0.359156,0.0,CRS2,4,XTOL_REACHED,18391,17.18,0.0,2.50048e7,6.86715e6,2.85743e6,9.72458e6,1200.0,3583.0,125.0,233.0,161.902,42948.9,0.0140035,0.0,6.87182e5,8.44215e5
8,2,6.68086,1.8,0.262366,1.90124,1.4231,16.9999,0.413641,0.0,CRS2,4,XTOL_REACHED,13535,12.936,0.0,2.82309e7,7.49706e6,4.29165e6,1.17887e7,2897.0,4820.0,309.0,350.0,39.6974,89869.8,0.0132549,0.160586,1.43792e6,9894.49
9,1,4.00108,1.8,5.48878,0.86583,1.53675,16.2703,0.359156,0.0,CRS2,5,XTOL_REACHED,17992,17.327,0.0,2.50048e7,6.86715e6,2.85743e6,9.72458e6,1200.0,3583.0,125.0,233.0,161.902,42948.9,0.0140035,0.0,6.87182e5,8.44215e5


In [28]:
CSV.write("raw_dataset_rule_based_comp.csv", df1)

"raw_dataset_rule_based_comp.csv"

In [34]:
@btime simulate_microgrid(X,capex_def,dispatch_1)

  630.200 μs (850 allocations: 2.44 MiB)


(Microgrid{Float64}(Project(25, 0.05, 1.0, "€"), [1453.0, 1331.0, 1214.0, 1146.0, 1078.0, 1035.0, 1032.0, 1072.0, 1142.0, 1208.0  …  1479.0, 1508.0, 1410.0, 1370.0, 1467.0, 1587.0, 1561.0, 1472.0, 1533.0, 1483.0], DispatchableCompound{Float64}(ProductionUnit{Float64}[ProductionUnit{Float64}(0.0, 0.0, 0.24, 1.0, 400.0, 0.02, 45000.0, 15000.0, 0.0, 1.0, 1.0, "L", "KWh")], ProductionUnit{Float64}[ProductionUnit{Float64}(1800.0, 0.0, 0.0625, 0.0, 1600.0, 45.0, 15.0, 45000.0, 0.05, 1.0, 1.0, "Kg", "KWh")]), ProductionUnit{Float64}[ProductionUnit{Float64}(2000.0, 0.0, 56.0, 0.0, 1600.0, 44.0, 20.0, 45000.0, 0.05, 1.0, 1.0, "KWh", "Kg")], TankCompound{Float64}(Tank{Float64}(0.0, 0.0, 0.0, 25.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0), Tank{Float64}(10000.0, 500.0, 4.0, 25.0, 0.0, 0.0, 0.0, 1.0, 14.0, 1.0, 1.0)), Battery{Float64}(5000.0, 350.0, 10.0, 15.0, 3000.0, 1.0, 1.0, 0.05, 0.0, 1.0, 0.0, 1.0, 1.0), NonDispatchableSource[Photovoltaic{Float64}(3000.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

## 2- sensitivity analysis
Given that the industrialisation of the manufacture of electrolysers and high-power PEM fuel cells is evolving, we have carried out a sensibility study on the investment prices of these components by considering two scenarios different from the base case described in section 2.2. Firstly, by considering the price reduction target set by the Fuel Cell and Hydrogen Joint Undertaking (FCH 2 JU) for 2030, referred to as "case 1". For this scenario, the investment prices of the electrolyser and the fuel cell will be €500/kW and €1,200/kW respectively. We then considered a scenario in which the prices of the electrolyser and the fuel cell would be higher for economic reasons, "case 2", at 2200€/kW and 2500€/kW respectively.

In [26]:
shed_max_list = [0.,0.0001,0.001]
capex_2= [400., 0.0, 2500., 500., 2200., 350., 1200., 3500.]
capex_1= [400., 0.0, 1200., 500., 500., 350., 1200., 3500.]
capexlist=[capex_1,capex_def,capex_2]
dispatchlist=[dispatch_1,dispatch_2]

2-element Vector{Function}:
 dispatch_1 (generic function with 2 methods)
 dispatch_2 (generic function with 1 method)

In [29]:
df2 = DataFrame( dispatch=Int64[],xopt_pv=Float64[],xopt_wind=Float64[] ,xopt_sto=Float64[],xopt_elyz=Float64[], xopt_fc=Float64[], xopt_hyTank=Float64[],
    lcoe_opt=Float64[], shed_max=Float64[],algo=String[],srand=Int[], ret=String[],numevals=Int64[],time=Float64[],shed_rate_opt=Float64[],
    capex=Float64[],om=Float64[],replacement=Float64[],opex=Float64[],
    fc_hours=Float64[],elyz_hours=Float64[],fc_starts=Float64[],elyz_starts=Float64[],batt_cycles=Float64[],h2_consumed=Float64[],ini_h2_level=Float64[],ini_bat_level=Float64[],
       fc_energy=Float64[],batt_energy=Float64[]
    
)

Row,dispatch,xopt_pv,xopt_wind,xopt_sto,xopt_elyz,xopt_fc,xopt_hyTank,lcoe_opt,shed_max,algo,srand,ret,numevals,time,shed_rate_opt,capex,om,replacement,opex,fc_hours,elyz_hours,fc_starts,elyz_starts,batt_cycles,h2_consumed,ini_h2_level,ini_bat_level,fc_energy,batt_energy
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Int64,String,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [30]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
maxeval=100000
xtol_rel=1e-8
srand=1
i=0;

for capex in capexlist,shed_max in shed_max_list, dispatch in dispatchlist
     t_ini= time()
    xopt, ret, numevals = optim_mg(x0,capex, shed_max,dispatch, algo, maxeval,xtol_rel,srand)
    topt = time() - t_ini
    mg,traj,stats,costs = simulate_microgrid(xopt,capex,dispatch)
    
    push!(df2,(findfirst(==(dispatch),dispatchlist),xopt[2],xopt[3], xopt[1],xopt[4], xopt[5], xopt[6],
            costs.lcoe,shed_max,"CRS2",srand,String(ret),numevals,topt,stats.shed_rate,
         costs.system.investment,costs.system.om,costs.system.replacement, costs.system.om+costs.system.replacement,stats.fc_hours,stats.elyz_hours,stats.fc_starts,stats.elyz_starts,stats.storage_cycles,
            stats.h2_consumed, mg.tanks.h2Tank.ini_filling_ratio,mg.storage.SoC_ini,stats.fc_energy,stats.storage_dis_energy     
        ))
        i=i+1;
     @printf("%s algo: %s after %d iterations. %d \nx*=", algo, ret,numevals,i)
println(round.(xopt*1000; digits=6)) # kW
end

GN_CRS2_LM algo: XTOL_REACHED after 24028 iterations. 1 
x*=[4420.856897, 4009.471979, 1800.0, 1123.880534, 1536.749701, 16501.056843]
GN_CRS2_LM algo: XTOL_REACHED after 21277 iterations. 2 
x*=[148.011585, 5696.058302, 1799.999999, 2296.9656, 1489.390715, 17727.304364]
GN_CRS2_LM algo: XTOL_REACHED after 36678 iterations. 3 
x*=[4417.244982, 4009.941421, 1800.0, 1123.029324, 1439.767199, 16469.829062]
GN_CRS2_LM algo: XTOL_REACHED after 15057 iterations. 4 
x*=[99.358623, 5572.285555, 1800.0, 2378.47129, 1420.464213, 17831.653607]
GN_CRS2_LM algo: XTOL_REACHED after 21278 iterations. 5 
x*=[4305.05989, 4050.773521, 1799.999999, 1122.561477, 1251.649034, 16181.888937]
GN_CRS2_LM algo: XTOL_REACHED after 14880 iterations. 6 
x*=[101.578605, 5594.469803, 1800.0, 2345.552147, 1249.101116, 17545.361782]
GN_CRS2_LM algo: XTOL_REACHED after 16719 iterations. 7 
x*=[5488.787623, 4001.076852, 1800.0, 865.828862, 1536.749704, 16270.322139]
GN_CRS2_LM algo: XTOL_REACHED after 13607 iterations. 

In [28]:
CSV.write("dataset_sensitivity.csv", df2)


"dataset_sensitivity.csv"

In [ ]:
df_raw = DataFrame(CSV.File("raw_dataset_rule_based_comp.csv"))

In [ ]:
df_Average = DataFrame( dispatch=Int64[],xopt_pv=Float64[],xopt_wind=Float64[] ,xopt_sto=Float64[],xopt_elyz=Float64[], xopt_fc=Float64[], xopt_hyTank=Float64[],
    lcoe_opt=Float64[], shed_max=Float64[],algo=String[],srand=Int[], ret=String[],numevals=Int64[],time=Float64[],shed_rate_opt=Float64[],
    capex=Float64[],om=Float64[],replacement=Float64[],opex=Float64[],
    fc_hours=Float64[],elyz_hours=Float64[],fc_starts=Float64[],elyz_starts=Float64[],batt_cycles=Float64[],h2_consumed=Float64[],ini_h2_level=Float64[],ini_bat_level=Float64[],
       fc_energy=Float64[],batt_energy=Float64[]
    
)